In [7]:
!nvidia-smi

Sun Jan 26 10:41:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import pandas as pd
from datasets import Dataset

# Load the dataset
df = pd.read_csv("cleaned_lang_data.csv")

# Remove duplicates if any
df = df.drop_duplicates()

# Convert the dataset to Hugging Face's Dataset format
hf_dataset = Dataset.from_pandas(df[['English', 'Japanese']])

# Train-Test split
hf_dataset = hf_dataset.train_test_split(test_size=0.2)
print(hf_dataset)


DatasetDict({
    train: Dataset({
        features: ['English', 'Japanese', '__index_level_0__'],
        num_rows: 218728
    })
    test: Dataset({
        features: ['English', 'Japanese', '__index_level_0__'],
        num_rows: 54683
    })
})


In [ ]:
# julianty/opus-tatoeba-en-ja-finetuned-eng-to-jpn_Hani

In [11]:
from transformers import MarianTokenizer

# Load tokenizer
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-tatoeba-en-ja")

# Tokenization function
def preprocess_function(batch):
    inputs = tokenizer(batch['English'], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(batch['Japanese'], max_length=128, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

# Apply tokenization to the dataset
tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/218728 [00:00<?, ? examples/s]

Map:   0%|          | 0/54683 [00:00<?, ? examples/s]

In [13]:
from transformers import MarianMTModel, Trainer, TrainingArguments

# Load the model
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-tatoeba-en-ja")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    fp16=True,  # Use mixed precision for faster training on GPUs
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-2ce8344e90b8>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.150100,0.139731
2,0.131100,0.128967
3,0.122400,0.125980


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=41013, training_loss=0.15019107415734076, metrics={'train_runtime': 7692.0448, 'train_samples_per_second': 85.307, 'train_steps_per_second': 5.332, 'total_flos': 2.224356469191475e+16, 'train_loss': 0.15019107415734076, 'epoch': 3.0})

In [14]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Model and tokenizer saved!")


Model and tokenizer saved!


In [16]:
from transformers import MarianTokenizer, MarianMTModel

# Load the fine-tuned model
model = MarianMTModel.from_pretrained("./fine_tuned_model")
tokenizer = MarianTokenizer.from_pretrained("./fine_tuned_model")

# Translation function
def translate_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Test the model
test_sentence = "I want to go japan."
translated_sentence = translate_sentence(test_sentence)
print(f"English: {test_sentence}")
print(f"Japanese: {translated_sentence}")


English: I want to go japan.
Japanese: 私はジャパンに行きたい。


In [29]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Fine-tuned model and tokenizer saved locally!")

Fine-tuned model and tokenizer saved locally!


In [18]:
import shutil

# Compress the model folder into a zip file
shutil.make_archive("fine_tuned_model", "zip", "./fine_tuned_model")
print("Model zipped as 'fine_tuned_model.zip'")


Model zipped as 'fine_tuned_model.zip'


In [30]:
!pip install huggingface_hub

In [31]:
from huggingface_hub import login

login()  # This will prompt you to input your Hugging Face token


In [33]:
from huggingface_hub import HfApi, HfFolder

# Replace 'your-model-name' with your desired model name
model_name = "Patlu29/eng-jap_trans"

# Upload the model and tokenizer
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

print(f"Model uploaded to https://huggingface.co/{model_name}")

model.safetensors:   0%|          | 0.00/310M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model uploaded to https://huggingface.co/Patlu29/eng-jap_trans


# Reuse the Model Later

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Patlu/eng-jap_trans"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Test the model
sentence = "I love learning languages."
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
